## Aufgabe 1 $\frac{e}{m}$-Bestimmung mit dem Fadenstrahlrohr
### 1.1 Untersuchung des Feldes eines Helmholtzspulenpaars 
Bauen Sie vor dem Plexiglaskasten mit Helmholtzspulen und Fadenstrahlrohr die zusätzliche Helmholtzspule mit Meßplatte so auf, dass sich die Meßplatte in der Mittelebene zwischen zwei Spulen befindet. Die Anordnung gleicht der Spulenanordnung, in der sich das Fadenstrahlrohr befindet. Die Spulen beiderseits der Meßplatte sind in Reihe zu schalten. Die Anschlüsse der betreffenden Spule im Plexiglaskasten sind an extra Buchsen herausgeführt. Die Meßplatte
kann an etlichen vorgesehenen Stellen die Hallsonde aufnehmen.\
Bild dieses Aufbaus:\
![](Helmholtz-Spulenpaar.jpg)  \
Messen Sie die Hallspannungen an den vorgesehenen Stellen für die Spulenströme $1.0$; $1.5$; und $2.0 A$.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from kafe2 import  XYContainer, Fit, XYFit, Plot, ContoursProfiler                            
import PhyPraKit as ppk
from scipy.signal import find_peaks
from uncertainties import ufloat, unumpy
from math import log10, floor

In [ ]:
data=np.loadtxt('1.1hallspannungen_ströme.txt', skiprows=4) #importiere Daten
U_1A=data[:,0]*10**(-3)
U_15A=data[:,1]*10**(-3)
U_2A=data[:,2]*10**(-3)
print('gemessene Hallspannungen bei einem Spulenstrom von 1A:', U_1A, 'in V')
print('gemessene Hallspannungen bei einem Spulenstrom von 1.5A:', U_15A, 'in V')
print('gemessene Hallspannungen bei einem Spulenstrom von 2A:', U_2A, 'in V')

### 1.2 Kalibrieren Sie die Hallsonde mit Hilfe des berechenbaren Feldes der langen Eichspule
Messen Sie etwa 10 Wertepaare (Hallspannung/Spulenstrom) und berechnen Sie daraus die Eichgerade B(UHall). Der Bereich der hier gemessenen Hallspannungen sollte zu den bei Aufgabe 1.1 gemessenen Werten passen.

In [ ]:
data=np.loadtxt('1.2hallspannungen_ströme.txt', skiprows=7) #importiere gemessene Daten
U_hall=data[:,1]*10**(-3)
I_sp=data[:,0]
print('angelegter Spulenstrom:',I_sp, 'in A')
print('gemessene Hallspannung:',U_hall, 'in V')

$B_{UHall}$ berechnen mit folgender Formel: $B_{UHall}=\frac{\mu_{0} \cdot I\cdot N}{L}$ \
Wobei N die Anzahl der Windungen, L die Länge der Spule und $\mu_{0}$ die magnetische Feldkonstante mit einem Wert von $\mu_{0}=1.256637 \cdot 10^{-6} \frac{N}{A^2}$ ist.

In [ ]:
y0=1.256637*10**(-6)
L= 0.3
N=750
B_hall=(y0*I_sp*N)/L
print(U_hall,B_hall)
#indexes = [0, 6, 7] # Entfernen 0,6,7 für experimentelle bestimmung
#B_hall = np.delete(B_hall, indexes)
#U_hall = np.delete(U_hall, indexes)
#print(B_hall)
#print(U_hall)

In [ ]:
xy_data = XYContainer(U_hall,B_hall)

line_fit=Fit(data=xy_data)
line_fit.do_fit()
line_fit.assign_parameter_names(x='U_H', a='a', b='B_0')
line_fit.assign_model_function_expression('B')
line_fit.assign_model_function_expression("{a} * {x} + {b}")
line_fit.report()

line_fit.assign_parameter_latex_names(x='U_H', a='a', b='B_0')
line_fit.assign_model_function_latex_name('B')
line_fit.assign_model_function_latex_expression('{a} \\cdot {x} + {b}')


line_fit.data_container.label = "Verwendete Daten"
line_fit.model_label = "Modellfunktion"

line_fit.data_container.axis_labels = ["U_H in V", "B in T"]

plot = Plot(fit_objects=line_fit)
plot.plot()
plot.show()
plot.save()

In [ ]:
B0=-2.1*10**(-5)
B1=0.031
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)
def B_hall(U, B0=-1.9*10**(-5), B1=0.031):
    return B0 + B1*U
print('Damit ergibt sich für die Eichgerade: B_hall(U)=',round_sig(B0,4),'+',round_sig(B1,4),'* U')

### 1.3 Vergleichen Sie den gemessenen Wert des Mittenfeldes zwischen den Helmholtzspulen mit dem berechneten Wert. Überprüfen Sie die Feldhomogenität bei der vorliegenden Spulenanordnung. 
Unter der Voraussetzung Spulenradius = Spulenabstand = R gilt für das Mittenfeld $$B=0.7155\cdot \mu_{0} \cdot n \cdot \frac{I}{R}$$
Dabei ist n die Windungszahl der Spule und I der Strom in der Spule.

In [ ]:
B_mess=B_hall
y0=1.256637*10**(-6)
n=130
R=0.15
def B_rech(I, y0, n, R):
    return 0.7155*y0*n*I/R


In [ ]:
indexes = [0, 6, 7] # Entfernen 0,6,7 für experimentelle bestimmung
B_1A_mess=B_mess(U_1A)
print(B_1A_mess)
B_1A_mess = np.delete(B_1A_mess, indexes)
print(B_1A_mess)
B_1A_mess_mean=np.mean(B_1A_mess)
B_1A_mess_abw=np.sqrt(np.var(B_1A_mess))
B_1A_mess_u=ufloat(B_1A_mess_mean,B_1A_mess_abw)
B_rech(1, y0, n, R)
print('Für die gemessene Stromstärke von 1A ergibt sich ein durschnittliches B-Feld mit folgender Stärke:',B_1A_mess_u,' T aus den Berechnungen ergibt sich:',B_rech(1, y0, n, R),'T') 

B_15A_mess=B_mess(U_15A)
B_15A_mess = np.delete(B_15A_mess, indexes)
B_15A_mess_mean=np.mean(B_15A_mess)
B_15A_mess_abw=np.sqrt(np.var(B_15A_mess))
B_15A_mess_u=ufloat(B_15A_mess_mean,B_15A_mess_abw)
B_rech(1.5, y0, n, R)
print('Für die gemessene Stromstärke von 15A ergibt sich ein durschnittliches B-Feld mit folgender Stärke:',B_15A_mess_u,' T aus den Berechnungen ergibt sich:',B_rech(1.5, y0, n, R),'T')

B_2A_mess=B_mess(U_2A)
B_2A_mess = np.delete(B_2A_mess, indexes)
B_2A_mess_mean=np.mean(B_2A_mess)
B_2A_mess_abw=np.sqrt(np.var(B_2A_mess))
B_2A_mess_u=ufloat(B_2A_mess_mean,B_2A_mess_abw)
B_rech(2, y0, n, R)
print('Für die gemessene Stromstärke von 2A ergibt sich ein durschnittliches B-Feld mit folgender Stärke:',B_2A_mess_u,' T aus den Berechnungen ergibt sich:',B_rech(2, y0, n, R),'T')

### 1.4

In [ ]:
data=np.loadtxt('1.4a).txt', skiprows=3) #importiere gemessene Daten
U_1A=data[:,0]
R_1A=data[:,1]*10**(-2)
U_2A=data[:,2]
R_2A=data[:,3]*10**(-2)

plt.plot(U_1A, R_1A, c='red', lw=1, label='I=1A')
plt.plot(U_2A, R_2A, c='blue', lw=2, label='I=2A' )
plt.xlabel('U in V')
plt.ylabel('R in m')
plt.legend()
plt.title('Graphische Darstellung Daten aus 1.4a)')
plt.show()

In [ ]:
data=np.loadtxt('1.4b).txt', skiprows=3) #importiere gemessene Daten
I_160V=data[:,0]
R_160V=data[:,1]*10**(-2)
I_250V=data[:,2]
R_250V=data[:,3]*10**(-2)

plt.plot(I_160V, R_160V, c='red', lw=1, label='U=160V')
plt.plot(I_250V, R_250V, c='blue', lw=2, label='U=250V' )
plt.xlabel('I in A')
plt.ylabel('R in m')
plt.legend()
plt.title('Graphische Darstellung Daten aus 1.4b)')
plt.show()
